# Relationship Between GDP and Rental Prices

## Comparisons
1. Taking all the data points averaged by year and comparing to the national GDP.
1. Averaging data points by state and year and comparing to GDP by state.
1. Looking at Virginia cities, compare Virginia rental prices averaged by year to Virginia GDP.

## Method
1. Import Rent data
1. Create Rent DataFrame for each category: all cities, by state, and only for Virginia
1. Import yearlyGDPbyState csv data and apply inflation method
1. Add base year GDP data into each of the Rent DataFrames
1. Run visualizations to find any trends in the data

In [20]:
# import needed libraries
from numpy import *
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)

pd.set_option('display.max_columns', None)

## Read in the Zillow Monthly Rental Data 

In [21]:
# load the zillow rental csv into a dataframe
dfZRental = pd.read_csv(r'../DataSet/Zillow_Rental_2014-2020.csv',header=0,encoding = "ISO-8859-1")

#### Reusing Diana's pivot method from DataDictionary-ZillowRental.ipynb

In [22]:
#use melt to change monthly dates into rows - all data
dfZRentalPivot = dfZRental.melt(id_vars=['RegionID','RegionName', 'SizeRank'], var_name='YYYY-MM-DD')

dfZRentalPivot['yr'] = pd.DatetimeIndex(dfZRentalPivot['YYYY-MM-DD']).year
dfZRentalPivot['period'] = pd.DatetimeIndex(dfZRentalPivot['YYYY-MM-DD']).month

dfZRentalPivot

,RegionID,RegionName,SizeRank,YYYY-MM-DD,value,yr,period
0,102001,United States,0,2014-01,1349.0,2014,1
1,394913,"New York, NY",1,2014-01,2318.0,2014,1
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,2014-01,1856.0,2014,1
3,394463,"Chicago, IL",3,2014-01,1533.0,2014,1
4,394514,"Dallas-Fort Worth, TX",4,2014-01,1221.0,2014,1
5,394974,"Philadelphia, PA",5,2014-01,1370.0,2014,1
6,394692,"Houston, TX",6,2014-01,1278.0,2014,1
7,395209,"Washington, DC",7,2014-01,1891.0,2014,1
8,394856,"Miami-Fort Lauderdale, FL",8,2014-01,1592.0,2014,1
9,394347,"Atlanta, GA",9,2014-01,1154.0,2014,1


In [23]:
dfZRentalPivot[dfZRentalPivot['RegionName'] == 'United States']

,RegionID,RegionName,SizeRank,YYYY-MM-DD,value,yr,period
0,102001,United States,0,2014-01,1349.0,2014,1
106,102001,United States,0,2014-02,1354.0,2014,2
212,102001,United States,0,2014-03,1360.0,2014,3
318,102001,United States,0,2014-04,1365.0,2014,4
424,102001,United States,0,2014-05,1371.0,2014,5
530,102001,United States,0,2014-06,1376.0,2014,6
636,102001,United States,0,2014-07,1381.0,2014,7
742,102001,United States,0,2014-08,1387.0,2014,8
848,102001,United States,0,2014-09,1392.0,2014,9
954,102001,United States,0,2014-10,1398.0,2014,10
